In [ ]:
!pip install textstat textblob spacy streamlit openai python-dotenv
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm

In [ ]:
import os
from dotenv import load_dotenv
import textstat
from textblob import TextBlob
import spacy
import json

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
nlp = spacy.load("en_core_web_sm")

In [ ]:
bias_terms = {
    "disability_exclusion": ["handicapped", "crippled"],
    "immigration_sensitive": ["illegal alien", "illegals"],
    "gendered": ["chairman", "policeman"],
    "ageist": ["elderly people", "old people"]
}

In [ ]:
def score_readability(text):

    grade = textstat.text_standard(text, float_output=True)
    if grade <= 6:
        return 25
    elif grade <= 8:
        return 22
    elif grade <= 10:
        return 18
    elif grade <= 12:
        return 14
    else:
        return 8

def score_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity >= 0.4:
        return 15
    elif polarity >= 0.1:
        return 12
    elif polarity >= -0.1:
        return 10
    else:
        return 6

def score_bias(text):
    text_lower = text.lower()
    penalties = 0
    matches = []
    for category, terms in bias_terms.items():
        for t in terms:
            if t in text_lower:
                penalties += 5
                matches.append((category, t))
    score = max(25 - penalties, 5)
    return score, matches

def score_accessibility(text):

    sentences = list(nlp(text).sents)
    avg_len = sum(len(s.text.split()) for s in sentences) / max(len(sentences), 1)
    if avg_len <= 16:
        return 20
    elif avg_len <= 22:
        return 16
    elif avg_len <= 28:
        return 12
    else:
        return 8

def score_transparency(text):
    indicators = ["according to", "source:", "data from", "you should", "next steps", "to apply"]
    count = sum(1 for i in indicators if i in text.lower())
    if count >= 3:
        return 15
    elif count == 2:
        return 12
    elif count == 1:
        return 8
    else:
        return 5

In [ ]:
def evaluate_text(text):
    r = score_readability(text)
    s = score_sentiment(text)
    b, matches = score_bias(text)
    a = score_accessibility(text)
    t = score_transparency(text)
    total = r + s + b + a + t
    report = {
        "readability_score": r,
        "sentiment_score": s,
        "bias_score": b,
        "accessibility_score": a,
        "transparency_score": t,
        "total_score": total,
        "flagged_terms": matches
    }
    return report

sample_text = """
The agency will process your request. To apply, visit the official website. Seniors, immigrants and people with disabilities can request assistance.
"""
report = evaluate_text(sample_text)
report

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

def improve_text_for_civic_use(text):
    prompt = f"""
    You are an AI assistant helping a state government team write accessible, bias-free responses for the public.
    Rewrite the following text in plain English, avoid exclusionary or outdated terms, and keep it friendly and actionable.

    Text:
    {text}
    """
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user", "content": prompt}],
        temperature=0.3
    )
    return resp.choices[0].message.content.strip()

In [ ]:
import streamlit as st

st.title("FAIR-Lens – Responsible AI Output Auditor")

user_text = st.text_area("Paste AI-generated text here", height=200)

if st.button("Evaluate"):
    report = evaluate_text(user_text)
    st.subheader("Audit Report")
    st.json(report)
    if report["total_score"] < 70:
        st.warning("Score below 70 – consider rewriting for accessibility or bias.")
        if OPENAI_API_KEY:
            improved = improve_text_for_civic_use(user_text)
            st.subheader("Suggested Rewrite")
            st.write(improved)